In [ ]:
# @title ## Εισαγωγή βιβλιοθηκών
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.stattools import adfuller
from scipy.stats import kruskal
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from prophet import Prophet


In [ ]:
# @title ## Εισαγωγή συνόλου δεδομένων

def load_data_and_print_sample(pathfile:str):
    '''
    Args:
      pathfile: The file to be used including the times-series data

    Output: dataframe with the time series
    '''

    df = pd.read_csv(pathfile, parse_dates=['date'], index_col='date')
    print(df.head(30))
    print("\n")
    print(f" the dataset's columns are: {list(df.columns)}")

    return df


pathfile = "/content/training_set.csv"
df = load_data_and_print_sample(pathfile)


# Χρησιμοποιώ δύο στιγμιότυπα του συνόλου δεδομένων:
 # 1. για ανάλυση και προβλέψεις με χρήση στατιστικών μοντέλων (ARIMA etc)
df_for_statistics = df.copy()

 # 2. για ανάλυση και προβλέψεις με χρήση  μοντέλων Machine/Deep Learning (LSTM, Prophet etc)
df_for_ml = df.copy()

In [ ]:
# @title Εμφανιση στατιστικών στοιχείων
df.describe()

In [ ]:
# @title Οπτικοποίηση χρονοσειράς
# Select only numeric columns
numeric_cols = df.select_dtypes(include='number').columns

# Plot each numeric column over time
plt.figure(figsize=(15, 8))
# Create a separate plot for each numeric column
for col in numeric_cols:
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df[col])
    plt.title(f"Time Series of {col}")
    plt.xlabel("Date")
    plt.ylabel(col)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# @title Κωδικοποίηση κατηγορικών μεταβλητών
#
encoder = LabelEncoder()
df['wnd_dir'] = encoder.fit_transform(df['wnd_dir'])

# Display the unique mappings
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("Label encoding mapping for 'wnd_dir':", label_mapping)

# Show the updated DataFrame
print(df.head(10))

In [ ]:
# @title Έλεγχος για Τάση με χρήση στατιστικού τέστ

# A. Δοκιμή Augmented Dickey-Fuller (ADF) για Στασιμότητα (αν υπάρχει τάση)
# Μηδενική υπόθεση: Η χρονοσειρά ΔΕΝ είναι στασιμή (δηλ. έχει τάση).
# Αν p-value < 0.05 → απορρίπτεται η μηδενική → η σειρά είναι στασιμή.

def adf_test(df, cols):
    # Αν είναι μία στήλη (string), το κάνουμε λίστα
    if isinstance(cols, str):
        cols = [cols]

    # Φιλτράρουμε μόνο τις αριθμητικές στήλες
    numeric_cols = df.select_dtypes(include='number').columns
    target_cols = [col for col in cols if col in numeric_cols]

    for col in target_cols:
        series = df[col].dropna()
        result = adfuller(series)

        print(f"Έλεγχος ADF για τη στήλη '{col}':")
        print(f"  Στατιστικό ADF: {result[0]:.4f}")
        print(f"  p-value: {result[1]:.4f}")
        if result[1] < 0.05:
            print("  ✅ Στασιμή σειρά (χωρίς τάση)")
        else:
            print("  ❌ Μη στασιμή σειρά (πιθανώς με τάση)")
        print("-" * 60)


adf_test(df, "pollution")

In [ ]:
# @title Έλεγχος για Εποχικότητα με χρήση στατιστικού τέστ


def kruskal_seasonality(df, freq='hour'):
    df_ = df.copy()
    df_['season'] = getattr(df_.index, freq)

    numeric_cols = df_.select_dtypes(include='number').columns
    for col in numeric_cols:
        groups = [group[col].dropna().values for name, group in df_.groupby('season')]
        stat, p = kruskal(*groups)
        print(f"Kruskal-Wallis για '{col}' κατά {freq}: p = {p:.4f}", "✅ Εποχικότητα" if p < 0.05 else "❌ Όχι εποχικότητα")

kruskal_seasonality(df)

In [ ]:
# @title Εντοπισμός Εποχικότητας με χρήση διαγράμματος αυτοσυσχέτισης (autocorrelation plot)

def plot_acf_for_columns(df, cols, lags=50):

    if isinstance(cols, str):
        cols = [cols]

    for col in cols:
        if pd.api.types.is_numeric_dtype(df[col]):
            series = df[col].dropna()
            plt.figure(figsize=(10, 4))
            plot_acf(series, lags=lags)
            plt.title(f"ACF για τη στήλη '{col}'")
            plt.tight_layout()
            plt.show()
        else:
            print(f"❌ Η στήλη '{col}' δεν είναι αριθμητική.")

plot_acf_for_columns(df, 'temp')

In [ ]:
# @title Εξάλειψη εποχικότητας
def decompose_and_plot(series, name, freq):
    result = seasonal_decompose(series.dropna(), model='additive', period=freq)
    result.plot()
    plt.suptitle(f"Αποσύνθεση Χρονοσειράς: {name}", fontsize=12)
    plt.tight_layout()
    plt.show()


def remove_seasonality(df, period):
    df = df.copy()
    deseasoned_df = pd.DataFrame(index=df.index)
    numeric_cols = df.select_dtypes(include='number').columns
    for col in numeric_cols:
        try:
            # Αφαίρεση εποχικότητας μέσω αποσύνθεσης
            result = seasonal_decompose(df[col].dropna(), model='additive', period=period, extrapolate_trend='freq')
            deseasoned_series = df[col] - result.seasonal
            deseasoned_df[col] = deseasoned_series
        except Exception as e:
            print(f"⚠️ Δεν ήταν δυνατή η αποσύνθεση για τη στήλη '{col}': {e}")

    return deseasoned_df

decompose_and_plot(df['pollution'], 'Pollution', 1)
df_statistics_no_seasonality = remove_seasonality(df_for_statistics, 24)

print(df_statistics_no_seasonality)

In [ ]:
# @title Διαχωρισμός συνόλου δεδομένων και χαρακτηριστικού προς πρόβλεψη

def train_test_split_df_with_target(df, target_col='pollution', test_size=0.2):
    """
    Διαχωρίζει df σε X (features), y (target), και τα χωρίζει σε train/test
    """
    X = df.drop(columns=[target_col])
    y = df[target_col]

    split_point = int(len(df) * (1 - test_size))

    X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
    y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

    return X_train, X_test, y_train, y_test


In [ ]:
# @title Στατιστικά μοντέλα για πρόβλεψη: ARIMA, ExponentialSmoothing
def arima_model(series):
    return ARIMA(series, order=(2, 0, 2))

def ets_model(series):
    return ExponentialSmoothing(series, trend='add', seasonal=None, damped_trend=True)


In [ ]:
# @title Συνάρτηση για την αξιολόγηση μετρικών για κάθε στατιστικό μοντέλο

def evaluate_forecasting_model(model_func, y_train, y_test, model_name):
    model = model_func(y_train)
    fit = model.fit()
    forecast = fit.forecast(steps=len(y_test))

    # Μετρικές
    mse = mean_squared_error(y_test, forecast)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, forecast)

    print(f"📊 {model_name}")
    print(f"  MSE  : {mse:.3f}")
    print(f"  RMSE : {rmse:.3f}")
    print(f"  R²   : {r2:.3f}")
    print("-" * 40)

    return forecast, mse, rmse, r2


In [ ]:
# @title Συνάρτηση για την οπτικοποίηση αποτελεσμάτων
def plot_forecasts(y_test, forecasts, labels):
    plt.figure(figsize=(12, 5))
    plt.plot(y_test.index, y_test.values, label='Actual', linewidth=2)

    for forecast, label in zip(forecasts, labels):
        plt.plot(y_test.index, forecast, label=label)

    plt.title('Προβλέψεις με Στατιστικά Μοντέλα')
    plt.xlabel('Ημερομηνία')
    plt.ylabel('Ρύπανση (pollution)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# @title Εφαρμογή μοντέλων
# Εκπαίδευση / Έλεγχος
X_train_stat, X_test_stat, y_train_stat, y_test_stat = train_test_split_df_with_target(df_statistics_no_seasonality)
print(X_train_stat.shape, y_train_stat.shape)
print(X_test_stat.shape, y_test_stat.shape)

# Μοντέλα και εκτίμηση
forecast_arima, mse_a, rmse_a, r2_a = evaluate_forecasting_model(arima_model, y_train_stat, y_test_stat, 'ARIMA')
forecast_ets, mse_e, rmse_e, r2_e = evaluate_forecasting_model(ets_model, y_train_stat, y_test_stat, 'ETS')

# Οπτικοποίηση
plot_forecasts(y_test_stat, [forecast_arima, forecast_ets], ['ARIMA', 'ETS'])


In [ ]:
# @title ## Κανονικοποίηση δεδομένων για προβλέψεις με τη χρήση αλγορίθμων μηχανικής μάθησης
# Drop non-numeric columns
# Keep only numeric columns
df_ml_numeric = df_for_ml.select_dtypes(include=[np.number])

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_ml_numeric)
df_ml_scaled = pd.DataFrame(scaled_data, columns=df_ml_numeric.columns, index=df_ml_numeric.index)


In [ ]:
# @title ## Long Short-Term Memory  (LSTM) Model

# Select the target variable (e.g., 'temp')
target_col = "pollution"
target_data = df_ml_scaled[[target_col]]

# Create sequences
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(target_data.values)

# Train-test split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X, y, test_size=0.2, shuffle=False)

# LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_ml.shape[1], X_train_ml.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train_ml, y_train_ml, epochs=20, batch_size=32, validation_data=(X_test_ml, y_test_ml), verbose=1)

# Predict
y_pred = model.predict(X_test_ml)

# Inverse scale predictions for interpretation
y_test_inv = scaler.inverse_transform(np.hstack([np.zeros((len(y_test_ml), df_ml_numeric.shape[1]-1)), y_test_ml.reshape(-1, 1)]))[:, -1]
y_pred_inv = scaler.inverse_transform(np.hstack([np.zeros((len(y_pred), df_ml_numeric.shape[1]-1)), y_pred]))[:, -1]

# Plot
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label="Actual")
plt.plot(y_pred_inv, label="Predicted")
plt.title("LSTM Forecasting - Temperature")
plt.xlabel("Time step")
plt.ylabel("Temperature")
plt.legend()
plt.show()



In [ ]:
# @title  ## Prophet Model

# --- PREPARE DATA FOR PROPHET ---
df_prophet = df[[target_col]].reset_index().rename(columns={"date": "ds", target_col: "y"})

# Split manually based on time
split_point = int(len(df_prophet) * 0.8)
df_train_prophet = df_prophet.iloc[:split_point]
df_test_prophet = df_prophet.iloc[split_point:]

# --- FIT PROPHET ---
prophet_model = Prophet(daily_seasonality=True)
prophet_model.fit(df_train_prophet)

# Forecast same length as test
future = prophet_model.make_future_dataframe(periods=len(df_test_prophet), freq='H')
forecast = prophet_model.predict(future)

# --- GET FORECASTED VALUES ---
y_test_prophet = df_test_prophet['y'].values
y_pred_prophet = forecast.iloc[-len(df_test_prophet):]['yhat'].values

# --- METRICS (PROPHET) ---
mse_prophet = mean_squared_error(y_test_prophet, y_pred_prophet)
rmse_prophet = np.sqrt(mse_prophet)
r2_prophet = r2_score(y_test_prophet, y_pred_prophet)

print("📊 Prophet Evaluation:")
print(f"  MSE  : {mse_prophet:.2f}")
print(f"  RMSE : {rmse_prophet:.2f}")
print(f"  R²   : {r2_prophet:.3f}")
print("-" * 40)

# --- PLOT PROPHET RESULTS ---
plt.figure(figsize=(12, 6))
plt.plot(y_test_prophet, label="Actual")
plt.plot(range(len(y_test_prophet)-200, len(y_test_prophet)), y_pred_prophet[-200:], label="Prophet Forecast (last 200)")
plt.title("📈 Prophet Forecast - Pollution")
plt.xlabel("Time Step")
plt.ylabel("Pollution")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()
